In [1]:
# As first_reg, but with features and labels

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = False

In [4]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass

In [5]:
### DNN hyperparameters
hidden_units = [2000] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4
preprocessor = ScaleDataOverall()
uuid = None

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 500  # How many steps to print the current loss.
model_dir_base = './saved_models/multi_peakon/'
model_dir = model_dir_str(model_dir_base, hidden_units, logits, drop_rate, 
                          preprocessor, batch_norm=False, uuid=uuid)

dnn = create_dnn(hidden_units, logits, drop_rate=drop_rate,
                 model_dir=model_dir, log_steps=log_steps,
                 gradient_clip=gradient_clip, batch_norm=batch_norm)
preprocessor.load(model_dir)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall_DUP1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4c24d65710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.


In [6]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 30
    steps = 3000
    train_input_fn = preprocessor.data(BatchData(batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])
    
    batch_size = 300
    steps = 6000
    train_input_fn = preprocessor.data(BatchData(batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

In [7]:
poly_deg = 10
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (dnn, preprocessor.training(False), 'DNN'),
                           (BilinearInterp(), None, 'Bilinear'),
                           (PolyInterp(poly_deg), None, 'Poly-{}'.format(poly_deg))],
                          X_peak,
                          y_peak)

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall_DUP1/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [8]:
results[1].loss, results[2].loss, results[3].loss

(28.226852844668944, 196.58519956744044, 107.33592336933178)

In [9]:
dnns_pros = create_dnns_pros_from_dir(model_dir_base)

INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/seventh_reg/1000x10_121_D04_ScaleDataByBatch', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4c24dcefd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/seventh_reg/1000x10_121_D04_ScaleDataByBatch_DUP1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoi

In [10]:
average_reg = RegressorAverager(dnns_pros)
average_reg.set_mask([False, False, True, True, True, True, True])
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (average_reg, None, 'RegressorAverager')],
                          X_peak,
                          y_peak)

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/1000x10_121_D04_ScaleDataByBatch_DUP2/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/1500x10_121_D04_ScaleDataByBatch/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/1500x10_121_D04_ScaleDataByBatch_DUP1/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done ca

In [11]:
results[1].loss

1.9421526361565138